## Build an Optimal Diversified Portfolio Based on My Stock Picks

After careful research I picked 13 stocks I would like to invest in. The question now is how will I split my money between these stocks.
My 'optimal' portfolio should meet the following conditions:
- Annual expected Return >12% including dividends
- Includes all stock picks, and no single stock >15% than my portfolio
- Technology stock should be<40% the total size of my portfolio
- Average Beta <1
- I want the portfolio to have the highest return at the lowest beta


In [6]:
import pandas as pd
import random
url="https://raw.githubusercontent.com/tareklel/Stock-Picker/master/Stock%20Spreadsheet%20-%20Sheet1.csv"
stocks=pd.read_csv(url)
#only select stock I picked
stocks=stocks[stocks["Yes/No"]=="Yes"].reset_index()

In [69]:
print(stocks[["Stock","Expected Annual Growth in Earning","Beta"]])

            Stock Expected Annual Growth in Earning  Beta
0            Nike                             13.30  0.67
1       Accenture                              9.00  1.10
2          Boeing                             15.90  1.29
3          Disney                              4.30  1.02
4            BB&T                             12.50  1.14
5             J&J                              8.20  0.67
6   United Health                             10.70  0.80
7            Visa                             12.30  1.00
8          Amazon                             30.30  1.77
9       Microsoft                             10.80  1.21
10         Google                             16.00  1.04
11     Salesforce                             17.00  1.38
12        Verizon                              7.90  0.49


In [7]:
#Format percentage from str to float
stocks['Expected Annual Growth in Earning']=stocks['Expected Annual Growth in Earning'].str.replace("%","")
stocks['Annual Dividend Income']=stocks['Annual Dividend Income'].str.replace("%","")

Let's say I have a 1000USD to split between my 13 stocks
I want to create 10^5 random iterations of splitting a 1000 into 13 intervals such that 
no stock is less than 2% and greater than 15% of the total portfolio

In [84]:
combo_list=[]
for a in range(10000):
    split=[]
    for cubicle in range(12):
        #20/1000 is 2% while 150/1000 is 15% so integer should be between 20 and 150 
        cubicle=random.randint(20, 150)
        split.append(cubicle)
    #The final integer is the difference between all previous integer and 1000    
    split.append(1000-sum(split))
    #Filter out splits that don't meet our conditions including negative integers in the 13th slot
    if split[12]>10 and split[12]<150:
        split=[x/1000 for x in split]
        #Shuffle splits in case there will be a bias in the 13th slot since it was created differently
        random.shuffle(split)
        #Make sure tech stock <40% of portfolio
        if (split[8]+split[9]+split[10]+split[11])<401:
            combo_list.append(split)

In [85]:
matrix=[]
for combo in combo_list:
    #multiply each split by the each stock's expected growth to get the cumilative growth 
    #and then multiply the split by beta to get the beta mean
    #save the growth and beta into a list
    #iterate for each split
    minimatrix=[]
    df = pd.Series(combo)
    #Format integers to floats
    stocks["Expected Growth"]=stocks['Expected Annual Growth in Earning'].astype('float64')\
    +stocks['Annual Dividend Income'].astype('float64')
    stocks["Expected Growth"]=df.multiply(stocks['Expected Growth'].astype('float64'))
    #Shorten the floats to the third decimal point
    minimatrix.append(float("{0:.3f}".format(sum(stocks["Expected Growth"]))))
    minimatrix.append(float("{0:.3f}".format(sum(df.multiply(stocks["Beta"])))))
    matrix.append(minimatrix)
#How many possible solutions do we have
print(len(matrix))
    

3209


In [102]:
#Select Min Growth and Max Beta
minimum_growth,max_beta=12,.9

optimal_solution_found=False

for index in range(len(matrix)):
    if matrix[index][0]>minimum_growth and matrix[index][1]<max_beta:
        minimum_growth=matrix[index][0]
        max_beta=matrix[index][1]
        winning_index=index
        optimal_solution_found=True        
if optimal_solution_found==True:
    print("optimal solution found at index {}".format(winning_index))
    print("expected growth is {}%, expected beta is {}".format(matrix[winning_index][0],matrix[winning_index][1]))
elif optimal_solution_found==False:
    #Create additional list in combo_list with zeroes and assign its index to winning index
    #In case someone chooses the index and uses it on the stock table the answers will be zero
    null_list=[]
    combo_list.append([null_list.append(0) for x in range(13)])
    winning_index=len(combo_list)-1
    print("optimal solution not found")



optimal solution found at index 2632
expected growth is 12.636%, expected beta is 0.885


In [103]:
stocks["Optimal Stock Distribution"]=combo_list[winning_index]
print(stocks[["Stock","Optimal Stock Distribution"]])

            Stock  Optimal Stock Distribution
0            Nike                       0.148
1       Accenture                       0.034
2          Boeing                       0.027
3          Disney                       0.141
4            BB&T                       0.032
5             J&J                       0.142
6   United Health                       0.074
7            Visa                       0.115
8          Amazon                       0.031
9       Microsoft                       0.025
10         Google                       0.063
11     Salesforce                       0.035
12        Verizon                       0.133
